In [6]:
#imports
import ipywidgets as wg
from SearchEngine import SearchEngine
from ManipulatePyExercise import ManipulatePyExercise

class AppInterface:
    
    #constructor
    def __init__(self):
        self.startVariables()
        self.displaySearchEngine()
    
    #create and initialize variables
    def startVariables(self):
        #search
        self.searchEngine = SearchEngine()
        self.directory = self.searchEngine.getFileDirectory()
        self.searchEngine.setDirectory(self.directory)
        
        #exercise
        self.exercise = ManipulatePyExercise()
        
        #window size
        self.num_percent = 80
        self.percent = str(self.num_percent)+'%'
        
        #widgets
        self.buttonBackSearch = 0
        self.textViewSearch = 0
        self.buttonSearch = 0
        self.BoxSearch = wg.Box([])
        
        self.BoxFolders = wg.Box([])
        self.BoxExercise = wg.Box([])
        self.BoxVersions = wg.Box([])
        self.BoxDifferences = wg.Box([])
        self.BoxPyDifferences = wg.Box([])
        self.BoxTexDifferences = wg.Box([])
        self.BoxSeeVersions = wg.Box([])
        self.BoxSeeVersion = wg.Box([])
        
        #versions
        self.versions = 0
        self.versionError = wg.HTML()
        self.errorversion = False
    
    #____________________________________________________________________________________________________SEARCH
    def getDirectory(self):
        return self.searchEngine.getDirectory()
    
    def setDirectory(self, directory):
        self.searchEngine.setDirectory(directory)
    
    #create and diplay interface widgets
    def displaySearchEngine(self):
        self.BoxSearch.close() #fechar a anterior se necessário
        #------------------------------------------------------------------------------------button Back Search
        buttonBackSearch_layout = wg.Layout(flex='1 20 auto', 
                                        width='50px')
        buttonBackSearch_layout = wg.Layout(flex='1 20 auto',
                                                width='auto')
        self.buttonBackSearch = wg.Button(description='<', tooltip='back', layout=buttonBackSearch_layout)
        self.buttonBackSearch.style.button_color = 'lightblue'
        self.buttonBackSearch.on_click(self.back)
        #---------------------------------------------------------------------------------------textView Search
        textViewSearch_layout = wg.Layout(flex='15 20 auto', 
                                          width='auto')
        self.textViewSearch = wg.Text(value=str(self.getDirectory()), layout=textViewSearch_layout)
        #-----------------------------------------------------------------------------------------button Search
        buttonSearch_layout = wg.Layout(flex='1 20 auto', 
                                        width='auto')
        self.buttonSearch = wg.Button(description='Search', layout=buttonSearch_layout, button_style='info')
        self.buttonSearch.on_click(self.search)
        #--------------------------------------------------------------------------------------------Box Search
        searchBox_layout = wg.Layout(display='flex',
                                    flex_flow='row',
                                    align_items='stretch',
                                    width=self.percent)
        self.BoxSearch = wg.Box(children=[self.buttonBackSearch,
                                          self.textViewSearch, 
                                          self.buttonSearch], 
                                        layout=searchBox_layout)
        #------------------------------------------------------------------------------------DISPLAY Box Search
        display(self.BoxSearch)
        self.displayFoldersBox()
    
    def closeBoxSerach(self):
        self.BoxSearch.close()

    #search onClick action
    def search(self, b):
        if(self.textViewSearch.value != ""):
            self.directory = self.textViewSearch.value
        self.searchEngine.setDirectory(self.directory) #alterar diretoria
        self.textViewSearch.value = self.directory #alterar texto da busca
        self.displayFoldersBox() #display folders
    
    #back search
    def back(self, b):
        self.searchEngine.goBack()
        self.refreshTextViewPath()
        self.displayFoldersBox()
    
    #foward search
    def fowardSearch(self, b):
        self.searchEngine.goFoward(b.description)
        self.refreshTextViewPath()
        self.verifyFolder()
    
    #create and display folders
    def displayFoldersBox(self):
        self.BoxFolders.close() #fechar a anterior se necessário
        folders_layout = wg.Layout(width='auto')
        folders = self.searchEngine.getFolders()
        filtered_folders = self.filterFolders(folders)
        items = [wg.Button(description=folder, layout=folders_layout) for folder in filtered_folders]
        for item in items:
            item.on_click(self.fowardSearch)
        BoxFiles_layout = wg.Layout(display='flex',
                                    flex_flow='column',
                                    align_items='stretch',
                                    width=self.percent)
        self.BoxFolders = wg.Box(children=items, layout=BoxFiles_layout)
        #------------------------------------------------------------------------------------DISPLAY Box Folders
        display(self.BoxFolders)
    
    def closeFoldersBox(self):
        self.BoxFolders.close()
    
    def refreshTextViewPath(self):
        self.directory = self.searchEngine.getDirectory()
        self.textViewSearch.value = self.directory

    def filterFolders(self, folders):
        #verificar se tem um ponto
        newFolders = []
        for folder in folders:
            if "." not in folder:
                if folder != 'qom_questions_transformer':
                    newFolders.append(folder)
        return newFolders
        
    def verifyFolder(self):
        #verificar se é um exercicio
        if(self.exercise.seeIfExercise(self.directory)):
            self.closeBoxSerach()#fechar search
            self.closeFoldersBox()#fechar folders 
            self.displayBoxExercise()#abrir exercicio
        else:#se não for continuar
            self.displayFoldersBox()
            
            
    #_____________________________________________________________________________________________SHOW EXERCISE
    
    def displayBoxExercise(self):
        #--------------------------------------------------------------------------------------------Button Back
        buttonb_layout = wg.Layout(width='50px')
        buttonBack = wg.Button(description='<', layout=buttonb_layout)
        buttonBack.style.button_color = 'lightblue'
        buttonBack.on_click(self.backToSearch)
        #----------------------------------------------------------------------------------------Button Versions
        buttons_layout = wg.Layout()
        buttonVersions = wg.Button(description='Generate Versions', layout=buttons_layout)
        buttonVersions.on_click(self.displayBoxVersions)
        #--------------------------------------------------------------------------------------Button Diferences
        buttonDiferences = wg.Button(description='Versions Differences', layout=buttons_layout)
        buttonDiferences.on_click(self.displayBoxDifferences)
        #------------------------------------------------------------------------------------Button See Versions
        buttonSeeVersions = wg.Button(description='See Versions', layout=buttons_layout)
        buttonSeeVersions.on_click(self.displayBoxSeeVersions)
        #--------------------------------------------------------------------------------------------Box Buttons
        buttonsBox_layout = wg.Layout(display='flex',
                                       flex_flow='row',
                                       align_items='stretch')
        if self.exercise.seeIfHasVersions():
            boxButtons = wg.Box(children=[buttonBack,
                                         buttonVersions,
                                         buttonDiferences,
                                         buttonSeeVersions],
                                      layout=buttonsBox_layout)
        else:
            boxButtons = wg.Box(children=[buttonBack,
                                          buttonVersions,
                                          buttonDiferences],
                                          layout=buttonsBox_layout)
        #-----------------------------------------------------------------------------------------Question Name
        #html
        name = self.exercise.getExerciseName()
        question_name = wg.HTML(value='</br> <p style="font-size:14px; color:#5200cc"><b>QUESTION: "'+name+'</b>"</p></br>')
        nameBox_layout = wg.Layout(display='flex')
        nameBox = wg.VBox(children=[question_name],
                                layout=nameBox_layout)
        #---------------------------------------------------------------------------------------------Box Image
        #ir buscar enunciado
        img = self.exercise.getExerciseImg()
        exerciseImage = wg.Image(value=img,format='png', width=self.percent)
        imageBox_layout = wg.Layout(display='flex')
        boxImage = wg.VBox(children=[exerciseImage],
                                   layout=imageBox_layout)
        #---------------------------------------------------------------------------------------------Box Lines
        #ir buscar alineas
        lines = self.exercise.getExerciseLinesImg()
        box = []
        for i in range((int)(len(lines)/2)): #para cada alinea
            #adicionar alinea
            n = i+1
            alinea = wg.HTML(value="</br> <b>Answer " + str(n) + ":</b>")
            box.append(alinea)
            #adicionar o tipo e a resposta
            type_true = wg.HTML(value="<p style='color:#248f24'> <b>True</b> </p>")
            answer_true = wg.Image(value=lines[i*2],format='png')
            type_false = wg.HTML(value="<p style='color:#e60000'> <b>False</b> </p>")
            answer_false = wg.Image(value=lines[i*2+1],format='png')
            line_layout = wg.Layout(display='flex',
                                    flex_flow='row')
            lineBox_true = wg.VBox(children=[answer_true, type_true],
                             layout = line_layout)
            box.append(lineBox_true)
            lineBox_false = wg.VBox(children=[answer_false, type_false],
                             layout = line_layout)
            box.append(lineBox_false)
            
            
        linesBox_layout = wg.Layout(display='flex', width=self.percent)
        boxLines = wg.VBox(children=box,
                           layout=linesBox_layout)
        #-------------------------------------------------------------------------------------------Box Exercise
        exerciseBox_layout = wg.Layout(display='flex',
                                       align_items='stretch',
                                       width=self.percent)
        self.BoxExercise = wg.VBox(children=[boxButtons,
                                             nameBox,
                                             boxImage,
                                             boxLines],
                                             layout=exerciseBox_layout)
        #-----------------------------------------------------------------------------------DISPLAY Box Exercise
        display(self.BoxExercise)
        
    def closeBoxExercise(self):
        self.BoxExercise.close()
        
    def backToSearch(self, b):
        self.closeBoxExercise()
        self.back(0)
        self.displaySearchEngine()
        
        
    #_______________________________________________________________________________________________SEE VERSIONS
    def displayBoxSeeVersions(self, b):
        self.closeBoxExercise()
        #--------------------------------------------------------------------------------------------Button Back
        button_layout = wg.Layout(width='50px')
        buttonBack = wg.Button(description='<', layout=button_layout)
        buttonBack.style.button_color = 'lightblue'
        buttonBack.on_click(self.backToExercise)
        #---------------------------------------------------------------------------------------------Choose text
        text_choose = wg.HTML('<p style="font-size:14px;"> Choose the version you would like to see: </p>')
        #-----------------------------------------------------------------------------------------------Versions
        #ir buscar o nome das pastas com versões
        versions_names = self.exercise.getVersionsNames()
        #criar butões para cada pasta
        button_layout = wg.Layout(width='auto')
        items = [wg.Button(description=name, layout=button_layout) for name in versions_names]
        for item in items:
            item.on_click(self.displayTheVersion)
            item.style.button_color = 'AliceBlue'
        Box_layout = wg.Layout(display='flex',
                               align_items='stretch',
                               width=self.percent)
        box_version_names = wg.VBox(children = items, layout = Box_layout)
        #---------------------------------------------------------------------------------------Box see versions
        self.BoxSeeVersions = wg.VBox(children = [buttonBack,
                                                  text_choose,
                                                  box_version_names],
                                        layout = Box_layout)
        #----------------------------------------------------------------------------------------------DISPLAY
        display(self.BoxSeeVersions)
    
    def displayTheVersion(self, b):
        version = b.description[-1]
        #--------------------------------------------------------------------------------------------Button Back
        button_layout = wg.Layout(width='50px')
        buttonBack = wg.Button(description='<', layout=button_layout)
        buttonBack.style.button_color = 'lightblue'
        buttonBack.on_click(self.backToBoxSeeVersions)
        #-----------------------------------------------------------------------------------------Question Name
        #html
        name = self.exercise.getExerciseName()
        question_name = wg.HTML(value='</br> <p style="font-size:14px; color:#5200cc"><b>QUESTION: "'+name+'", version '+version+'</b></p></br>')
        nameBox_layout = wg.Layout(display='flex')
        nameBox = wg.VBox(children=[question_name],
                                layout=nameBox_layout)
        #---------------------------------------------------------------------------------------------Box Image
        #ir buscar imagens
        imgs = self.exercise.getVersionImgs(version)
        exerciseImage = wg.Image(value=imgs[0],format='png', width=self.percent)
        imageBox_layout = wg.Layout(display='flex')
        boxImage = wg.VBox(children=[exerciseImage],
                                   layout=imageBox_layout)
        #---------------------------------------------------------------------------------------------Box Lines
        #ir buscar alineas
        box = []
        for i in range((int)(len(imgs[1])/2)): #para cada alinea
            #adicionar alinea
            n = i+1
            alinea = wg.HTML(value="</br> <b>Answer " + str(n) + ":</b>")
            box.append(alinea)
            #adicionar o tipo e a resposta
            type_true = wg.HTML(value="<p style='color:#248f24'> <b>True</b> </p>")
            answer_true = wg.Image(value=imgs[1][i*2],format='png')
            type_false = wg.HTML(value="<p style='color:#e60000'> <b>False</b> </p>")
            answer_false = wg.Image(value=imgs[1][i*2+1],format='png')
            
            line_layout = wg.Layout(display='flex',
                                    flex_flow='row')
            lineBox_true = wg.VBox(children=[answer_true, type_true],
                             layout = line_layout)
            box.append(lineBox_true)
            lineBox_false = wg.VBox(children=[answer_false, type_false],
                             layout = line_layout)
            box.append(lineBox_false)
            
            
        linesBox_layout = wg.Layout(display='flex', width=self.percent)
        boxLines = wg.VBox(children=box,
                           layout=linesBox_layout)
        #-------------------------------------------------------------------------------------------Box Exercise
        Box_layout = wg.Layout(display='flex',
                                       align_items='stretch',
                                       width=self.percent)
        self.BoxSeeVersion = wg.VBox(children=[buttonBack,
                                               nameBox,
                                               boxImage,
                                               boxLines],
                                               layout=Box_layout)
        #----------------------------------------------------------------------------------------------DISPLAY
        self.closeBoxSeeVersions()
        display(self.BoxSeeVersion)
    
    def closeBoxSeeVersion(self):
        self.BoxSeeVersion.close()
    
    def backToBoxSeeVersions(self, b):
        self.closeBoxSeeVersion()
        self.displayBoxSeeVersions(0)
        
    def closeBoxSeeVersions(self):
        self.BoxSeeVersions.close()
        
    #___________________________________________________________________________________________SHOW DIFFERENCES
    def displayBoxDifferences(self, b):
        self.closeBoxExercise()
        #--------------------------------------------------------------------------------------------Button Back
        button_layout = wg.Layout(width='50px')
        buttonBack = wg.Button(description='<', layout=button_layout)
        buttonBack.style.button_color = 'lightblue'
        buttonBack.on_click(self.backToExercise)
        #---------------------------------------------------------------------------------------------Choose text
        text_choose = wg.HTML('<p style="font-size:14px;"> Choose the file you would like to see: </p>')
        #------------------------------------------------------------------------------------------Py differences
        #ir buscar os ficheiros py existentes
        pyFiles = self.exercise.getPyFilesNames()
        #criar os botões para cada py file
        buttonPy_layout = wg.Layout(width='auto')
        itemsPy = [wg.Button(description=pyFile, layout=buttonPy_layout) for pyFile in pyFiles]
        for item in itemsPy:
            item.on_click(self.displayPyDiferencesBox)
            item.style.button_color = 'Gold'
        boxPy_layout = wg.Layout(display='flex',
                                    flex_flow='column',
                                    align_items='stretch',
                                    width=self.percent)
        boxPy = wg.Box(children=itemsPy, layout=boxPy_layout)
        #-----------------------------------------------------------------------------------------Tex differences
        #ir buscar os ficheiros tex existentes
        texFiles = self.exercise.getTexFilesNames()
        #criar os butões para cada tex file
        buttonTex_layout = wg.Layout(width='auto')
        itemsTex = [wg.Button(description=texFile, layout=buttonPy_layout) for texFile in texFiles]
        for item in itemsTex:
            item.on_click(self.displayTexDifferencesBox)
            item.style.button_color = 'Plum'
        boxTex_layout = wg.Layout(display='flex',
                                    flex_flow='column',
                                    align_items='stretch',
                                    width=self.percent)
        boxTex = wg.Box(children=itemsTex, layout=boxTex_layout)
        #-----------------------------------------------------------------------------------------Box Differences
        differencesBox_layout = wg.Layout(display='flex',
                                       align_items='stretch',
                                       width=self.percent)
        self.BoxDifferences = wg.VBox(children = [buttonBack,
                                                  text_choose,
                                                  boxPy,
                                                  boxTex],
                                      layout = differencesBox_layout)
        #----------------------------------------------------------------------------------DISPLAY Box Diferences
        display(self.BoxDifferences)
    
    def closeBoxDifferences(self):
        self.BoxDifferences.close()
    
    def backToExercise(self, b):
        self.closeBoxDifferences()
        self.colseBoxVersions()
        self.closeBoxSeeVersions()
        self.displayBoxExercise()
        
    def displayPyDiferencesBox(self, b):
        #------------------------------------------------------------------------------------Box Py Differences
        #ir buscar o nome do ficheiro 
        fileName = b.description
        #------------------------------------------------------------------------------------------Button back
        button_layout = wg.Layout(width='50px')
        buttonBack = wg.Button(description='<', layout=button_layout)
        buttonBack.style.button_color = 'lightblue'
        buttonBack.on_click(self.backToDifferences)
        #----------------------------------------------------------------------------------------------Py Text
        fileNameText = wg.HTML('<p style="font-size:18px; color:#0059b3">'+fileName+'</p></br>')
        text = wg.HTML('<div style="font-family:Courier New; font-size:14px">'+self.exercise.getPyFileDifferences(fileName)+'</div>')
        #----------------------------------------------------------------------------------------- Box Py Text
        boxPy_layout = wg.Layout(display='flex',
                                   align_items='stretch',
                                   border='solid')
        boxPy = wg.VBox(children = [fileNameText,
                                    text],
                        layout = boxPy_layout)

        differencesPyBox_layout = wg.Layout(display='flex',
                                            align_items='stretch',
                                            width=self.percent)
        self.BoxPyDifferences = wg.VBox(children = [buttonBack,
                                                    boxPy],
                                layout = differencesPyBox_layout)
        #---------------------------------------------------------------------------------------DISPLAY Box Py
        self.closeBoxDifferences()
        display(self.BoxPyDifferences)
    
    def closeBoxPyDifferences(self):
        self.BoxPyDifferences.close()
    
    def displayTexDifferencesBox(self, b):
        #-----------------------------------------------------------------------------------Box Tex Differences
        #ir buscar o nome do ficheiro 
        fileName = b.description
        #------------------------------------------------------------------------------------------Button back
        button_layout = wg.Layout(width='50px')
        buttonBack = wg.Button(description='<', layout=button_layout)
        buttonBack.style.button_color = 'lightblue'
        buttonBack.on_click(self.backToDifferences)
        #----------------------------------------------------------------------------------------------Tex Text
        fileNameText = wg.HTML('<p style="font-size:18px; color:#0059b3">'+fileName+'</p></br>')
        text = wg.HTML('<div style="font-family:Courier New; font-size:14px">'+self.exercise.getTexFileDifference(fileName)+'</div>')
        #----------------------------------------------------------------------------------------- Box Tex Text
        boxTex_layout = wg.Layout(display='flex',
                                   align_items='stretch',
                                   border='solid')
        boxTex = wg.VBox(children = [fileNameText,
                                    text],
                        layout = boxTex_layout)

        differencesTexBox_layout = wg.Layout(display='flex',
                                            align_items='stretch',
                                            width=self.percent)
        self.BoxTexDifferences = wg.VBox(children = [buttonBack,
                                                    boxTex],
                                layout = differencesTexBox_layout)
        #--------------------------------------------------------------------------------------DISPLAY Box Tex
        self.closeBoxDifferences()
        display(self.BoxTexDifferences)
    
    def closeBoxTexDifferences(self):
        self.BoxTexDifferences.close()
        
    def backToDifferences(self, b):
        self.closeBoxPyDifferences()
        self.closeBoxTexDifferences()
        self.displayBoxDifferences(0)
        
    #___________________________________________________________________________________________CREATE VERSIONS
    def displayBoxVersions(self, b):
        self.closeBoxExercise()
        self.createBoxVersions()
        
    def createBoxVersions(self):
        #--------------------------------------------------------------------------------------------Button Back
        button_layout = wg.Layout(width='50px')
        buttonBack = wg.Button(description='<', layout=button_layout)
        buttonBack.style.button_color = 'lightblue'
        buttonBack.on_click(self.backToExercise)
        #---------------------------------------------------------------------------------------- text Versions
        text_version = wg.HTML('<p style="font-size:14px;"> Choose the number of versions to generate:</p>')
        #------------------------------------------------------------------------------------ textView Versions
        self.versions = wg.IntText(value=1,disabled=False)
        #---------------------------------------------------------------------------------------button Versions
        buttongenerate_layout = wg.Layout()
        buttonGenerate = wg.Button(description='Generate', layout=buttongenerate_layout)
        buttonGenerate.style.button_color = 'Coral'
        buttonGenerate.on_click(self.generateVersions)
        #------------------------------------------------------------------------------------------------error
        self.versionError = wg.HTML('<p style="color:#990000;">Please generate one or more versions.</p>')
        #------------------------------------------------------------------------------------------box Versions
        versionsBox_layout = wg.Layout(display='flex',
                                       align_items='stretch',
                                       width=self.percent)
        self.BoxVersions = wg.VBox(children=[buttonBack,
                                             text_version,
                                             self.versions,
                                             buttonGenerate],
                                      layout=versionsBox_layout)
        #----------------------------------------------------------------------------------DISPLAY Box Versions
        display(self.BoxVersions)
    
    def colseBoxVersions(self):
        self.versionError.close()
        self.BoxVersions.close()
    
    def generateVersions(self, b):
        number = self.versions.value
        if number >= 1:
            #alterar numero de vesões a gerar no ficheiro py
            self.exercise.changeNumberOfVersions(number)
            #gerar versões
            self.exercise.generateVersions()
            #voltar para exercicio
            self.colseBoxVersions()
            self.displayBoxExercise()
        else:
            if self.errorversion == False:
                display(self.versionError)
                self.errorversion = True
        
        
if __name__ == '__main__':
    AppInterface()
    

Box(children=(Button(description='<', layout=Layout(flex='1 20 auto', width='auto'), style=ButtonStyle(button_…

Box(children=(Button(description='DataBase', layout=Layout(width='auto'), style=ButtonStyle()), Button(descrip…

Box(children=(Button(description='true_or_false_question_32_bit_nums', layout=Layout(width='auto'), style=Butt…

Box(children=(Button(description='DataBase', layout=Layout(width='auto'), style=ButtonStyle()), Button(descrip…

In [3]:
from IPython.display import HTML

HTML('''<script>
$('div.input').hide();
</script>''')